In [ ]:
# %pip uninstall envirocar-py -y
# %pip install overpass

In [ ]:
# %load_ext autoreload
# %autoreload 2

# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx

import requests
import json
from urllib.parse import urljoin

from algorithms import mm_utils

# class HttpFailedException(Exception):
#     """ General exception raised whenever communication failes """

# class NotFoundException(HttpFailedException):
#     """ Exception raised whenever a certain resource has not been found """

# class NotAuthorizedException(HttpFailedException):
#     """ Exception raised whenever the user doesn't has the rights to access a certain resource """

# class MailNotConfirmedException(HttpFailedException):
#     """ Exception raised whenever the user hasn't confirmed his mail """

# class ECConfig:

#     class _ECConfig:
#         CONFIG_PARAMS = {
#             "ec_base_url": "https://envirocar.org/api/stable/",
#             "ec_username": "",
#             "ec_password": "",
#             "number_of_processes": 4
#         }

#         def __init__(self):
#             self.load_config()

#         def load_config(self):
#             for key, value in self.CONFIG_PARAMS.items():
#                 setattr(self, key, value)

#     _instance = None

#     def __init__(self):
#         if not ECConfig._instance:
#             ECConfig._instance = self._ECConfig()
#         for item in ECConfig._instance.CONFIG_PARAMS:
#             setattr(self, item, getattr(ECConfig._instance, item))
            
# class BboxSelector:
#     """ Class for creating bounding box requests """

#     def __init__(self, bbox: [float]):
#         self.min_x = bbox[0]
#         self.min_y = bbox[1]
#         self.max_x = bbox[2]
#         self.max_y = bbox[3]

#     @property
#     def lower_left(self):
#         return self.min_x, self.min_y

#     @property
#     def upper_right(self):
#         return self.max_x, self.max_y

#     @property
#     def param(self):
#         return { 'bbox': f'{self.min_x},{self.min_y},{self.max_x},{self.max_y}' }

# class TimeSelector:
#     def __init__(self, start_time=None, end_time=None):
#         self.start_time = start_time
#         self.end_time = end_time

#     @property
#     def param(self):
#         if self.start_time and self.end_time==None:
#             return { 'after': f'{self.start_time}' }
#         elif self.start_time==None and self.end_time:
#             return { 'before': f'{self.end_time}' }
#         elif self.start_time and self.end_time:
#             return { 'during': f'{self.start_time},{self.end_time}' }


# class RequestParam:
#     def __init__(self, path: str, method="GET", headers=None, params=None):
#         self.path = path
#         self.method = method
#         self.headers = headers or []
#         self.params = params or []
        
# def handle_error_status(http_download):
#     """Decorating function for handling enviroCar HTTP exceptions
    
#     Arguments:
#         http_download {function} -- function to wrap
    
#     Raises:
#         HttpFailedException: wrapper for client exceptions
#         exception: other exceptions
    
#     Returns:
#         function -- decorated function
#     """
#     def decorate(self, request: RequestParam):
#         try:
#             return http_download(self, request)
#         except requests.HTTPError as exception:
#             if exception.response.status_code < requests.codes['internal_server_error']:
#                 if exception.response.status_code == requests.codes['unauthorized']:
#                     message = "Unauthorized to access: {}".format(request.path)
#                     raise NotAuthorizedException(message)
#                 else: 
#                     message = "Failed to download from: {}".format(request.path)
#                     raise HttpFailedException(message) from exception
#             raise exception from exception
#     return decorate

# class DownloadClient:

#     def __init__(self, *, config=None):
#         self.config = config or ECConfig()

#     def download(self, download_requests, decoder=None):
#         if (isinstance(download_requests, RequestParam)):
#             download_requests = [download_requests]

#         with concurrent.futures.ThreadPoolExecutor(max_workers=self.config.number_of_processes) as executor:
#             download_list = [executor.submit(self._download, request) for request in download_requests]

#         result_list = []
#         for future in download_list:
#             try:
#                 decoded_data = future.result().decode('utf-8')
#                 result_list.append(decoded_data)
#             except HttpFailedException as e:
#                 warnings.warn(str(e))
#                 result_list.append(None)

#         if decoder:
#             return decoder(result_list)
#         return result_list

#     @handle_error_status
#     def _download(self, download_request: RequestParam):
#         url = urljoin(self.config.ec_base_url, download_request.path)
#         print('self.config.ec_base_url', self.config.ec_base_url)
#         print('download_request', download_request.path)
        
#         # set BasicAuth parameters
#         auth = None
#         if self.config.ec_username and self.config.ec_password:
#             auth = HTTPBasicAuth(self.config.ec_username, self.config.ec_password)

#         response = requests.request(
#             download_request.method,
#             url= url,
#             auth=auth,
#             headers=download_request.headers,
#             params=download_request.params
#         )

#         response.raise_for_status()
#         # LOG.info("Successfully downloaded %s", url)
#         return response.content

# class TrackAPI():
#     """Handles the API access to the enviroCar backend and returns queried results as dataframes"""
#     TRACKS_ENDPOINT = "tracks"
#     TRACK_ENDPOINT = "tracks/{}"
#     USERTRACKS_ENDPOINT = "users/{}/tracks"

#     def __init__(self, api_client=None):
#         self.api_client = api_client or DownloadClient()

#     def get_tracks(self, username=None, bbox:BboxSelector=None, time_interval:TimeSelector=None, num_results=10, page_limit=100):
#         """Handles queries against the enviroCar api

#         Keyword Arguments:
#             username {str} -- the username to request tracks for (default: {None})
#             bbox {BboxSelector} -- bbox query parameter (default: {None})
#             time_interval {TimeSelector} -- time interval query parameter (default: {None})
#             num_results {int} -- maximum number of tracks to request (default: {10})
#             page_limit {int} -- page limit (default: {100})

#         Returns:
#             GeoDataFrame -- A GeoDataFrame containing the tracks matching the request parameters
#         """
#         path = self._get_path(username=username)

#         # creating download_requests
#         download_requests = []
#         current_results = 0
#         current_page = 1
#         while current_results < num_results:
#             request_params = {
#                 'limit': page_limit,
#                 'page': current_page
#             }
#             if bbox:
#                 request_params.update(bbox.param)

#             if time_interval:
#                 request_params.update(time_interval.param)

#             request = RequestParam(path=path, params=request_params)
#             download_requests.append(request)

#             current_results += page_limit
#             current_page += 1

#         # request for /tracks
#         tracks_meta_df = self.api_client.download(download_requests, decoder=_parse_tracks_list_df)
#         tracks_meta_df = tracks_meta_df[:num_results]

#         if not tracks_meta_df.empty:
#             ids = tracks_meta_df['track.id'].values
#             tracks_df = self._get_tracks_by_ids(ids)
#             return tracks_df

#         return gpd.GeoDataFrame()

#     def get_track(self, track_id: str):
#         return self.api_client.download(
#             RequestParam(path=self._get_path(trackid=track_id)),
#             decoder=_parse_track_df)

#     def _get_tracks_by_ids(self, ids: [str]):
#         download_requests = [RequestParam(path=self._get_path(trackid=id)) for id in ids]
#         return self.api_client.download(download_requests, decoder=_parse_track_df)

#     def _get_path(self, *, username=None, trackid=None):
#         if username is None and trackid is None:
#             return self.TRACKS_ENDPOINT
#         if username:
#             return self.USERTRACKS_ENDPOINT.format(username)
#         if trackid:
#             return self.TRACK_ENDPOINT.format(trackid)

# def _parse_tracks_list_df(tracks_jsons):
#     if not isinstance(tracks_jsons, list):
#         tracks_jsons = [tracks_jsons]

#     tracks_meta_df = pd.DataFrame()
#     for tracks_json in tracks_jsons:
#         if tracks_json:
#             ec_data = json.loads(tracks_json)
#             df = pd.json_normalize(ec_data, 'tracks')
#             df.rename(columns=__rename_track_columns, inplace=True)
#             # tracks_meta_df = tracks_meta_df.append(df)
#             tracks_meta_df = tracks_meta_df.add(df)

#     return tracks_meta_df

# def _parse_track_df(track_jsons):
#     if not isinstance(track_jsons, list):
#         track_jsons = [track_jsons]

#     tracks_df = gpd.GeoDataFrame()
#     for track_json in track_jsons:
#         # read properties
#         car_df = pd.json_normalize(json.loads(track_json)['properties'])
#         car_df.columns = car_df.columns.str.replace('sensor.properties.', 'sensor.')
#         car_df.rename(columns=__rename_track_columns, inplace=True)

#         # read geojson values
#         track_df = gpd.read_file(track_json)
#         track_df = track_df.join(pd.json_normalize(track_df['phenomenons'])).drop(['phenomenons'], axis=1)

#         # combine dataframes
#         car_df = pd.concat([car_df]*len(track_df.index), ignore_index=True)
#         # tracks_df = tracks_df.append(track_df.join(car_df))
#         tracks_df = tracks_df.add(track_df.join(car_df))

#     return tracks_df

# def __rename_track_columns(x):
#     if not x.startswith('sensor'):
#         return 'track.' + x
#     return x
        
# config = ECConfig()
# client = DownloadClient(config=config)
# track_api = TrackAPI(client)
# tracks = track_api.get_tracks(num_results=4, page_limit=3)
# print(tracks)

url = "https://envirocar.org/api/stable/tracks/52f3836be4b0d8e8c27ed6f0"
response = requests.get(url)
data = json.loads(response.text)

# attributes = dir(data)
# for attribute in attributes:
#     print(attribute)
#
# print(data)

geometries = []
attributes = []

for feature in data['features']:
    geometry = shape(feature['geometry'])
    geometries.append(geometry)
    attributes.append(feature['properties'])

gdf = gpd.GeoDataFrame(data=attributes, geometry=geometries)

# Get the bounding box
bbox = gdf.total_bounds

# 'total_bounds' returns a tuple with (minx, miny, maxx, maxy) values
minx, miny, maxx, maxy = bbox

# Print the bounding box values
print("Min X:", minx)
print("Min Y:", miny)
print("Max X:", maxx)
print("Max Y:", maxy)

G=nx.Graph()

#座標を指定し地図データをダウンロード
G = ox.graph.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive')
fig, ax = ox.plot_graph(G, figsize = (16,16),show=False,close=False)

tripdata_edges = mm_utils.point_to_traj(gdf, columns = {'ele': 'average',
                                                                  'timestamp':'first',
                                                                  'sat':'first'})
tripdata_edges.plot(ax=ax, linewidth=3)

campus = ox.features.features_from_place('Somewhere', tags = {'name':True})
campus.plot(ax=ax, alpha=0.5)

# bbox_url = f'http://www.overpass-api.de/api/xapi?way[bbox={minx},{miny},{maxx},{maxy}][highway=*]'#.format(minx, miny, maxx, maxy)
# print(bbox_url)

# bbox_response = requests.get(bbox_url)
# print(bbox_response)
# attributes = dir(bbox_response)
# for attribute in attributes:
#     print(attribute)
    
# import overpass
# api = overpass.API(timeout=600)
# map_query = overpass.MapQuery(minx, miny, maxx, maxy)
# response = api.Get(map_query, responseformat="json")
# # response = api.get(f'node[bbox={minx},{miny},{maxx},{maxy}]')
# print(response)

# print(gdf)

# from envirocar import TrackAPI, DownloadClient, BboxSelector, ECConfig

# # create an initial but optional config and an api client
# config = ECConfig()
# track_api = TrackAPI(api_client=DownloadClient(config=config))



# bbox = BboxSelector([
#     7.601165771484375, # min_x
#     51.94807412325402, # min_y
#     7.648200988769531, # max_x
#     51.97261482608728  # max_y
# ])

# # issue a query
# track_df = track_api.get_tracks(bbox=bbox, num_results=50) # requesting 50 tracks inside the bbox
# track_df



In [3]:
# see the data structure of geopanda dataframe 
gdf.head()
gdf['phenomenons'][1]

{'GPS VDOP': {'value': 0.8, 'unit': 'precision'},
 'Intake Temperature': {'value': 31.0, 'unit': 'c'},
 'Rpm': {'value': 779.0, 'unit': 'u/min'},
 'GPS Accuracy': {'value': 11.0, 'unit': '%'},
 'GPS Bearing': {'value': 276.1000061035156, 'unit': 'deg'},
 'Throttle Position': {'value': 87.0, 'unit': '%'},
 'MAF': {'value': 9.359999656677246, 'unit': 'l/s'},
 'Speed': {'value': 5.0, 'unit': 'km/h'},
 'GPS Altitude': {'value': 102.0, 'unit': 'm'},
 'Intake Pressure': {'value': 101.0, 'unit': 'kPa'},
 'GPS PDOP': {'value': 1.2, 'unit': 'precision'},
 'Engine Load': {'value': 61.960784912109375, 'unit': '%'},
 'GPS Speed': {'value': 5.4, 'unit': 'km/h'},
 'GPS HDOP': {'value': 0.9, 'unit': 'precision'},
 'Calculated MAF': {'value': 12.734502114669114, 'unit': 'g/s'}}

In [4]:
# phenomenons contains trajectory information of each time point in dictionary format
# example of extracting phenomenons from first data point
dict = gdf['phenomenons'][0]

# create array that save trajectory information that we want to extract 
key_list = ['GPS Speed', 'GPS HDOP', 'GPS Bearing']

# Initialize data frame 
df = pd.DataFrame()

# loop to get trajectory infos for each point 
for key in key_list:
    temp = []
    for i in range(gdf.shape[0]):
        print(i)
        dict_temp = gdf['phenomenons'][i]
        if key in dict_temp:
            temp.append(dict_temp[key]['value'])
        else:
            temp.append(float("nan"))
    df[key] = temp


# combine geodata frame with the new extracted info    
gdf = pd.concat([gdf, df],axis = 1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81


In [7]:
# results of extracted info 
gdf 

,id,time,phenomenons,geometry,GPS Speed,GPS HDOP,GPS Bearing
0,52f3836be4b0d8e8c27ed6f2,2014-02-01T14:17:33Z,"{'GPS Bearing': {'value': 75.4000015258789, 'u...",POINT (7.65244 51.95425),1.8,1.5,75.400002
1,52f3836be4b0d8e8c27ed6f4,2014-02-01T14:17:43Z,"{'GPS VDOP': {'value': 0.8, 'unit': 'precision...",POINT (7.65227 51.95411),5.4,0.9,276.100006
2,52f3836be4b0d8e8c27ed6f6,2014-02-01T14:17:49Z,"{'Intake Temperature': {'value': 30.0, 'unit':...",POINT (7.65232 51.95411),2.7,1.0,77.500000
3,52f3836be4b0d8e8c27ed6f8,2014-02-01T14:17:55Z,"{'GPS VDOP': {'value': 0.8, 'unit': 'precision...",POINT (7.65241 51.95410),4.5,0.9,101.800003
4,52f3836be4b0d8e8c27ed6fa,2014-02-01T14:18:01Z,"{'MAF': {'value': 14.520000457763672, 'unit': ...",POINT (7.65257 51.95406),4.5,1.0,98.699997
...,...,...,...,...,...,...,...
77,52f3836be4b0d8e8c27ed78c,2014-02-01T14:25:16Z,"{'O2 Lambda Voltage': {'value': 0.0, 'unit': '...",POINT (7.63016 51.95977),0.0,1.1,NaN
78,52f3836be4b0d8e8c27ed78e,2014-02-01T14:25:22Z,"{'O2 Lambda Voltage': {'value': 0.0, 'unit': '...",POINT (7.63016 51.95977),0.0,1.0,NaN
79,52f3836be4b0d8e8c27ed790,2014-02-01T14:25:28Z,"{'GPS Accuracy': {'value': 11.0, 'unit': '%'},...",POINT (7.63024 51.95982),9.0,1.0,37.599998
80,52f3836be4b0d8e8c27ed792,2014-02-01T14:25:34Z,"{'GPS Bearing': {'value': 293.3999938964844, '...",POINT (7.63017 51.95981),2.7,1.2,293.399994


In [14]:
# save data frame 
gdf.to_pickle('envirocar.pkl')